In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm 
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
train_root = './data/train_preliminary/'
test_root = './data/test/'
ad = pd.read_csv(train_root+'ad.csv')
click_log = pd.read_csv(train_root +'click_log.csv')

user = pd.read_csv(train_root+'user.csv')
Tclick_log = pd.read_csv(test_root +'click_log.csv')
Tad = pd.read_csv(test_root +'ad.csv')

In [3]:
click_ad = pd.merge(click_log,ad, on = 'creative_id' )
click_ad 
# Tclick_ad = pd.merge(Tclick_log,Tad, on = 'creative_id' )
# Tclick_ad 

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319
...,...,...,...,...,...,...,...,...,...
30082766,52,309198,2459911,1,2119242,161,2,676,231
30082767,5,309198,452252,1,402185,2870,2,7158,247
30082768,76,309198,3686554,2,3172440,3979,2,52506,247
30082769,50,309198,2386973,1,2057811,1946,2,17757,259


In [4]:
stop_word = [37298,
  304118,
  32571,
  1779084,
  12838,
  1807219,
  2145432,
  2353377,
  47512,
  37295,
  2085566,
  649132,
  7400,
  11050,
  670,
  39714,
  599224,
  27216,
  69204,
  648402,
  4089042,
  24333,
  2361420,
  1662565,
  33944,
  599128,
  157148,
  311867,
  291979,
  114074,
  1216557,
  2716666,
  1145367,
  2297097,
  996428,
  71691,
  1359388,
  2145436,
  2361397,
  37605,
  40625,
  1216572,
  13069,
  228182,
  2716698,
  7823,
  441518,
  228215,
  230303,
  35568,
  1483247,
  80223,
  783522,
  626545,
  95238,
  2293978,
  1344,
  63440,
  391390,
  35391,
  174882,
  3714867,
  2281581,
  415805,
  2143574,
  11032,
  47567,
  1394668,
  214946,
  1401346,
  2027357,
  430586,
  32572,
  32384,
  214927,
  2577798,
  214241,
  1490726,
  71692,
  243181,
  304111,
  55527,
  3735057,
  24046,
  1371132,
  1583375,
  771243,
  182549,
  63441,
  3764119,
  1037290,
  19517,
  11053,
  5980,
  34452,
  441462,
  267775,
  1373697,
  1662670,
  4180378,
  2407050,
  515146,
  1328401,
  4097183,
  503749,
  47569,
  2679307,
  292026,
  33918,
  782930,
  96192,
  214243,
  960671,
  517906,
  3916180,
  14993,
  161121,
  63715,
  119460,
  40060,
  48676,
  53083,
  2915745,
  2759343,
  927675,
  2145152,
  190531,
  12916,
  1503096,
  210936,
  1073278,
  309573,
  70933,
  609050,
  3098564,
  81369,
  996537,
  819395,
  29259,
  415792,
  3504430,
  243890,
  2032927,
  1374579,
  746898,
  3072233,
  2293585,
  39906,
  3507883,
  1366123,
  1068338,
  244888,
  422565,
  32573,
  2256163,
  421447,
  1322019,
  71689,
  1584320,
  2694280,
  2524263,
  1493353,
  18461,
  5687,
  599770,
  66299,
  918536,
  739328,
  1779097,
  185172,
  133060,
  1366108,
  55692,
  1370659,
  70759,
  1471925,
  889098,
  739332,
  2027356,
  42577,
  3367633,
  215183,
  49574,
  3098567,
  3445744,
  147339,
  107103,
  71688,
  76874,
  2569455,
  569079,
  35392,
  60878,
  148785,
  214259,
  1365370,
  1505826,
  164502,
  132709,
  1073243,
  1366494,
  125836,
  754390,
  404574,
  2577794,
  305022,
  2220823,
  408850,
  447611,
  2556703,
  131899,
  2293419,
  1271112,
  194024,
  295581,
  421457,
  63439,
  599699,
  1621082,
  2147099,
  3606269,
  9148,
  409181,
  1695685,
  1441710,
  2220962,
  19521,
  68601,
  2163375,
  2312947,
  109156,
  1354702,
  19526,
  102153,
  210830,
  22013,
  696517,
  11052,
  187448,
  671535,
  4127507,
  769749,
  108656,
  1349326,
  927601,
  3053779,
  447625,
  2447,
  2823483,
  1355177,
  71850,
  60946,
  1698535,
  3948271,
  50244,
  96739,
  2979764,
  1241236,
  390704,
  2785305,
  15953,
  928362,
  1331651,
  1390120,
  11111,
  1143199,
  350647,
  19219,
  2982000,
  64667,
  961938,
  3024130,
  1447022,
  68598,
  69662,
  3032431,
  46445,
  1750832,
  927041,
  816224,
  57976,
  829025,
  666856,
  1242486,
  2544047,
  931713,
  2903811,
  1771,
  227865,
  1636057,
  2326748,
  1894115,
  30856,
  45067,
  1441689,
  50259,
  12236,
  96472,
  1143149,
  17100,
  1540530,
  814724,
  1390254,
  22182,
  2294021,
  2979731,
  3072140,
  3369601,
  2556697,
  681,
  1143180,
  3861855,
  2487813,
  214248,
  2482639,
  2850324,
  1104088,
  2996159,
  2508839,
  2361318,
  2353375,
  32566,
  72773,
  71690,
  1524047,
  764755,
  379902,
  1516293,
  2143576,
  1619148,
  102349,
  34453,
  11049,
  3135943,
  1908430,
  2577797,
  39817,
  1653054,
  96310,
  21092,
  3048896,
  1483376,
  3416667,
  75195,
  30210,
  927521,
  120916,
  33444,
  1111825,
  1401244,
  196062,
  1736232,
  1695654,
  2223999,
  1024894,
  146045,
  2361415,
  9794,
  2393248,
  625439,
  2342124,
  21852,
  119458,
  1350839,
  3684862,
  1832498,
  214469,
  2487811,
  599778,
  168570,
  1071851,
  122388,
  2979717,
  9426,
  1506880,
  381023,
  33917,
  11711,
  42161,
  1100760,
  14309,
  243680,
  157136,
  1169,
  2221084,
  2736461,
  135433,
  3443675,
  53893,
  3098593,
  455584,
  1559737,
  1902901,
  392044,
  72514,
  1042231,
  230844,
  2294086,
  1493222,
  2326746,
  2004885,
  2766802,
  1696925,
  1544792,
  2921558,
  2100129,
  2165482,
  2051670,
  1544592,
  96297,
  1352649,
  74258,
  69463,
  10258,
  1659000,
  2979817,
  1352637,
  32753,
  1116981,
  1540928,
  2978359,
  1700726,
  1441171,
  86332,
  52794,
  1079973,
  3321332,
  1638771,
  2261612,
  67985,
  21202,
  3421343,
  2419283,
  6417,
  1544749,
  870295,
  2165678,
  1681032,
  1436059,
  56470,
  92193,
  447694,
  674160,
  1818228,
  2499093,
  634817,
  2221056,
  148524,
  283271,
  120257,
  2959408,
  62397,
  481926,
  66318,
  217635,
  74646,
  3870707,
  1501695,
  600897,
  1501727,
  2361370,
  1493712,
  648212,
  42101,
  1249794,
  71687,
  626464,
  516403,
  98612,
  1995344,
  2914783,
  1626693,
  40675,
  3562358,
  136920,
  420825,
  2857148,
  3377613,
  12566,
  1934182,
  408960,
  63722,
  1859016,
  2407043,
  3123951,
  196069,
  214288,
  53616,
  1057513,
  620303,
  2313330,
  26598,
  2915744,
  97468,
  1695647,
  2597281,
  246789,
  262543,
  1902895,
  12563,
  440188,
  33935,
  68584,
  52666,
  3856576,
  726989,
  1111160,
  415810,
  50485,
  4065967,
  243891,
  200559,
  146787,
  1359341,
  188642,
  2959934,
  1140828,
  119545,
  739318,
  3071947,
  2556702,
  3527813,
  94024,
  45454,
  23833,
  1681016,
  302591,
  570298,
  1876526,
  26284,
  2701805,
  3072230,
  277101,
  1203283,
  650592,
  1073371,
  2500679,
  468174,
  310878,
  2286484,
  15086,
  421456,
  454069,
  3014720,
  70399,
  1382395,
  59050,
  7386,
  244736,
  1829725,
  2407068,
  2304235,
  448245,
  452051,
  1380720,
  11175,
  2147104,
  1067462,
  273715,
  513287,
  1567866,
  1323091,
  2217684,
  116356,
  2361439,
  2290670,
  68594,
  2716680,
  3146506,
  68589,
  614955,
  600920,
  2487812,
  1890383,
  747958,
  65035,
  1544848,
  45461,
  133764,
  1659021,
  3377371,
  3052115,
  1867961,
  1359323,
  1435072,
  103515,
  1424160,
  112773,
  61055,
  1473500,
  3291227,
  1401178,
  46018,
  642674,
  1544478,
  86337,
  339924,
  333267,
  915611,
  1340318,
  2382213,
  2542948,
  79897,
  7505,
  53955,
  77257,
  71647,
  48855,
  1366495,
  518831,
  609824,
  1085443,
  44862,
  1336021,
  2347281,
  53919,
  739324,
  225095,
  850251,
  2989648,
  2633988,
  46983,
  35657,
  179689,
  1174779,
  188898,
  1645218,
  3421353,
  48221,
  838581,
  3733221]

In [5]:
click_ad = click_ad[~click_ad['creative_id'].isin(stop_word)]

In [6]:
click_ad.shape

(25048289, 9)

In [4]:
###uni_col2
click_ad.sort_values(by='time',inplace=True)
click_ad['creative_id'] = click_ad['creative_id'].astype(str)
click_ad['product_category'] = click_ad['product_category'].astype(str)
click_ad['advertiser_id'] = click_ad['advertiser_id'].astype(str)
group0 = click_ad.groupby('user_id')
group1 = group0.industry.apply(list).reset_index()
group1

,user_id,industry
0,1,"[6, 106, 326, 326, \N, 326, 73, 217, 64, 245, ..."
1,2,"[318, 238, 6, 6, 318, 47, 317, 47, 6, 47, 242,..."
2,3,"[\N, \N, 322, 322, \N, 6, 54, 6, 322, 322, 205..."
3,4,"[238, 25, 248, \N, \N, \N, \N, \N, 88, 319, 6,..."
4,5,"[297, \N, 318, 6, 322, 6, 288, 322, 319, 238, ..."
...,...,...
899995,899996,"[54, 40, 54, 54, 40, 54, 40, 6, 202, 6, 248, 2..."
899996,899997,"[60, 5, 317, 60, 6, 289, 318, 297, 321, 161, 3..."
899997,899998,"[54, \N, \N, \N, 6, 6, 333, 6, 293, 6, 6, 6, 6..."
899998,899999,"[217, \N, 333, 317, 252, 6, 232, 317, 108, 6, ..."


In [5]:
ind = group1.industry.values
count = 0
for i in tqdm(range(ind.shape[0])):
    for j in range(len(ind[i])):
        if ind[i][j]=='\\N':
            try:
                ind[i][j]=ind[i][j-1]
            except:
                try:
                    k=j+1
                    while(ind[i][k]=='\\N'):
                        k+=1
                        
                    ind[i][j]=ind[i][k]
                except:
                    count+=1
count,group1

100%|██████████| 900000/900000 [00:11<00:00, 81181.02it/s]


(0,         user_id                                           industry
 0             1  [6, 106, 326, 326, 326, 326, 73, 217, 64, 245,...
 1             2  [318, 238, 6, 6, 318, 47, 317, 47, 6, 47, 242,...
 2             3  [319, 319, 322, 322, 322, 6, 54, 6, 322, 322, ...
 3             4  [238, 25, 248, 248, 248, 248, 248, 248, 88, 31...
 4             5  [297, 297, 318, 6, 322, 6, 288, 322, 319, 238,...
 ...         ...                                                ...
 899995   899996  [54, 40, 54, 54, 40, 54, 40, 6, 202, 6, 248, 2...
 899996   899997  [60, 5, 317, 60, 6, 289, 318, 297, 321, 161, 3...
 899997   899998  [54, 54, 54, 54, 6, 6, 333, 6, 293, 6, 6, 6, 6...
 899998   899999  [217, 217, 333, 317, 252, 6, 232, 317, 108, 6,...
 899999   900000  [147, 147, 183, 232, 88, 289, 291, 148, 73, 28...
 
 [900000 rows x 2 columns])

In [6]:
group2=group0.advertiser_id.apply(list).reset_index()
group3=group0.product_id.apply(list).reset_index()
group4=group0.product_category.apply(list).reset_index()
group = pd.merge(group2,group3,on='user_id')
group = pd.merge(group,group1,on='user_id')
group = pd.merge(group,group4,on='user_id')

In [8]:
adv = group.advertiser_id.values
pro = group.product_id.values
ind = group.industry.values
pry = group.product_category.values
group['uni_col2'] = group['advertiser_id']
uni = group.uni_col2.values
for i in tqdm(range(uni.shape[0])):
    for j in range(len(uni[i])):
        uni[i][j]= adv[i][j]+'_'+pro[i][j]+'_'+ind[i][j]+'_'+pry[i][j]
group

100%|██████████| 900000/900000 [01:01<00:00, 14538.73it/s]


,user_id,advertiser_id,product_id,industry,product_category,uni_col2
0,1,"[9702_136_6_2, 29455_\N_106_5, 7293_\N_326_5, ...","[136, \N, \N, \N, 1334, \N, \N, \N, \N, 64, 14...","[6, 106, 326, 326, 326, 326, 73, 217, 64, 245,...","[2, 5, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[9702_136_6_2, 29455_\N_106_5, 7293_\N_326_5, ..."
1,2,"[22885_87_318_2, 10686_80_238_2, 18562_129_6_2...","[87, 80, 129, 129, 87, \N, 1400, \N, 1261, \N,...","[318, 238, 6, 6, 318, 47, 317, 47, 6, 47, 242,...","[2, 2, 2, 2, 2, 18, 2, 18, 2, 18, 2, 18, 18, 2...","[22885_87_318_2, 10686_80_238_2, 18562_129_6_2..."
2,3,"[32974_36256_319_17, 9877_40905_319_17, 17018_...","[36256, 40905, 1674, 1674, 35985, \N, \N, 1031...","[319, 319, 322, 322, 322, 6, 54, 6, 322, 322, ...","[17, 17, 2, 2, 17, 18, 18, 2, 2, 2, 18, 12, 2,...","[32974_36256_319_17, 9877_40905_319_17, 17018_..."
3,4,"[19451_1862_238_2, 7976_\N_25_18, 13084_2625_2...","[1862, \N, 2625, 38743, 41265, 37758, 39904, 3...","[238, 25, 248, 248, 248, 248, 248, 248, 88, 31...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[19451_1862_238_2, 7976_\N_25_18, 13084_2625_2..."
4,5,"[11882_\N_297_5, 992_\N_297_8, 22885_87_318_2,...","[\N, \N, 87, 136, \N, 136, \N, \N, \N, 1064, \...","[297, 297, 318, 6, 322, 6, 288, 322, 319, 238,...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[11882_\N_297_5, 992_\N_297_8, 22885_87_318_2,..."
...,...,...,...,...,...,...
899995,899996,"[18103_\N_54_18, 13019_\N_40_18, 9934_\N_54_18...","[\N, \N, \N, \N, \N, \N, \N, 129, \N, 1567, 26...","[54, 40, 54, 54, 40, 54, 40, 6, 202, 6, 248, 2...","[18, 18, 18, 18, 18, 18, 18, 2, 5, 2, 2, 2, 2,...","[18103_\N_54_18, 13019_\N_40_18, 9934_\N_54_18..."
899996,899997,"[24952_26858_60_3, 24975_1377_5_2, 768_1400_31...","[26858, 1377, 1400, 26858, \N, \N, 87, \N, 125...","[60, 5, 317, 60, 6, 289, 318, 297, 321, 161, 3...","[3, 2, 2, 3, 18, 5, 2, 18, 2, 2, 2, 2, 2, 2, 2...","[24952_26858_60_3, 24975_1377_5_2, 768_1400_31..."
899997,899998,"[18103_\N_54_12, 43805_35420_54_17, 32974_3951...","[\N, 35420, 39519, 43559, \N, \N, \N, \N, \N, ...","[54, 54, 54, 54, 6, 6, 333, 6, 293, 6, 6, 6, 6...","[12, 17, 17, 17, 18, 18, 12, 18, 18, 18, 18, 1...","[18103_\N_54_12, 43805_35420_54_17, 32974_3951..."
899998,899999,"[19524_\N_217_5, 8126_35593_217_17, 14930_\N_3...","[\N, 35593, \N, 140, 2150, \N, \N, 140, \N, \N...","[217, 217, 333, 317, 252, 6, 232, 317, 108, 6,...","[5, 17, 5, 2, 13, 18, 5, 2, 5, 18, 18, 18, 18,...","[19524_\N_217_5, 8126_35593_217_17, 14930_\N_3..."


In [9]:
group = group[['user_id','uni_col2']]
group5=group0.creative_id.apply(list).reset_index()
group = pd.merge(group5,group,on='user_id' )
group =pd.merge(group,user,on='user_id' )
group


,user_id,creative_id,uni_col2,age,gender
0,1,"[209778, 877468, 821396, 1683713, 122032, 7169...","[9702_136_6_2, 29455_\N_106_5, 7293_\N_326_5, ...",4,1
1,2,"[63441, 155822, 39714, 609050, 441462, 1266180...","[22885_87_318_2, 10686_80_238_2, 18562_129_6_2...",10,1
2,3,"[661347, 808612, 593522, 710859, 825434, 72694...","[32974_36256_319_17, 9877_40905_319_17, 17018_...",7,2
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...","[19451_1862_238_2, 7976_\N_25_18, 13084_2625_2...",5,1
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[11882_\N_297_5, 992_\N_297_8, 22885_87_318_2,...",4,1
...,...,...,...,...,...
899995,899996,"[92193, 654526, 939409, 1477926, 1458800, 9315...","[18103_\N_54_18, 13019_\N_40_18, 9934_\N_54_18...",5,1
899996,899997,"[69204, 30210, 7400, 60330, 2804147, 3011222, ...","[24952_26858_60_3, 24975_1377_5_2, 768_1400_31...",3,2
899997,899998,"[648402, 1251649, 1370659, 2302153, 2915599, 1...","[18103_\N_54_12, 43805_35420_54_17, 32974_3951...",4,2
899998,899999,"[307407, 122834, 646020, 12838, 774777, 599699...","[19524_\N_217_5, 8126_35593_217_17, 14930_\N_3...",3,1


In [10]:
group['uni_col2'] = group['uni_col2'].apply(lambda x:' '.join(x))
group['creative_id'] = group['creative_id'].apply(lambda x:' '.join(x))
group

,user_id,creative_id,uni_col2,age,gender
0,1,209778 877468 821396 1683713 122032 71691 1940...,9702_136_6_2 29455_\N_106_5 7293_\N_326_5 1466...,4,1
1,2,63441 155822 39714 609050 441462 1266180 13069...,22885_87_318_2 10686_80_238_2 18562_129_6_2 25...,10,1
2,3,661347 808612 593522 710859 825434 726940 3920...,32974_36256_319_17 9877_40905_319_17 17018_167...,7,2
3,4,39588 589886 574787 1892854 1962706 2264105 12...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,5,1
4,5,296145 350759 24333 43235 852327 1054434 12964...,11882_\N_297_5 992_\N_297_8 22885_87_318_2 970...,4,1
...,...,...,...,...,...
899995,899996,92193 654526 939409 1477926 1458800 931537 181...,18103_\N_54_18 13019_\N_40_18 9934_\N_54_18 99...,5,1
899996,899997,69204 30210 7400 60330 2804147 3011222 24333 1...,24952_26858_60_3 24975_1377_5_2 768_1400_317_2...,3,2
899997,899998,648402 1251649 1370659 2302153 2915599 1779124...,18103_\N_54_12 43805_35420_54_17 32974_39519_5...,4,2
899998,899999,307407 122834 646020 12838 774777 599699 12879...,19524_\N_217_5 8126_35593_217_17 14930_\N_333_...,3,1


In [5]:
click_ad['unique_col']=unique_col
click_ad
click_ad.to_csv(train_root+'train_new_col_real.csv')

train_ori = pd.read_csv(train_root+'train_all.csv', index_col = 0)
click_ad.sort_values(by='time',inplace=True)
click_ad['unique_col'] = click_ad['unique_col'].astype(int).astype(str)

group0 = click_ad.groupby('user_id')
group1 = group0.unique_col.apply(list).reset_index()
group1['unique_col'] = group1['unique_col'].apply(lambda x:' '.join(x))
group1

group = pd.merge(train_ori,group1,on ='user_id' )
group

In [4]:
###adv_ind
click_ad.sort_values(by='time',inplace=True)
click_ad['product_category'] = click_ad['product_category'].astype(str)
click_ad['advertiser_id'] = click_ad['advertiser_id'].astype(str)
click_ad['uni_col'] = click_ad['advertiser_id']+'_'+ click_ad['product_id']+'_'+click_ad['industry']+'_'+click_ad['product_category']
click_ad['adv_pro'] =click_ad['product_id'] +'_'+ click_ad['advertiser_id']
click_ad['adv_ind'] =click_ad['industry'] +'_'+ click_ad['advertiser_id']
click_ad
# click_ad.sort_values(by='time',inplace=True)
# group = click_ad.groupby('user_id').adv_ind.apply(list).reset_index()
# group['adv_ind'] = group['adv_ind'].apply(lambda x:' '.join(x))
# train_ori = pd.read_csv(train_root+'train_all.csv', index_col = 0)
# group = pd.merge(train_ori,group,on ='user_id' )
# group

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,uni_col,adv_pro,adv_ind
11038879,1,43718,18612,1,19795,29,2,817,317,817_29_317_2,29_817,317_817
27435892,1,438200,1381,1,1987,26858,3,9717,60,9717_26858_60_3,26858_9717,60_9717
8170398,1,74183,59144,1,54883,\N,18,17627,54,17627_\N_54_18,\N_17627,54_17627
8170409,1,778236,59144,2,54883,\N,18,17627,54,17627_\N_54_18,\N_17627,54_17627
21227515,1,24882,35447,1,34332,\N,18,17969,27,17969_\N_27_18,\N_17969,27_17969
...,...,...,...,...,...,...,...,...,...,...,...,...
27628648,91,686189,4374309,1,3750661,3687,2,15863,247,15863_3687_247_2,3687_15863,247_15863
22613566,91,385266,1508104,1,1311397,30888,3,16713,81,16713_30888_81_3,30888_16713,81_16713
1297579,91,15032,2716698,1,2336903,1261,2,7115,6,7115_1261_6_2,1261_7115,6_7115
25951973,91,857796,4026938,1,3457224,\N,5,53531,73,53531_\N_73_5,\N_53531,73_53531


In [7]:
###industry 2
# click_ad.sort_values(by='time',inplace=True)
# group = click_ad.groupby('user_id').industry.apply(list).reset_index()
group3 =group0.industry.apply(list).reset_index()
group3['industry2'] = group3['industry']
ind = group3.industry2.values
count = 0
for i in tqdm(range(ind.shape[0])):
    for j in range(len(ind[i])):
        if ind[i][j]=='\\N':
            try:
                ind[i][j]=ind[i][j-1]
            except:
                try:
                    k=j+1
                    while(ind[i][k]=='\\N'):
                        k+=1
                        
                    ind[i][j]=ind[i][k]
                except:
                    count+=1
count   


group3['industry'] =group3['industry'].apply(lambda x:' '.join(x))
group3['industry2'] =group3['industry2'].apply(lambda x:' '.join(x))
del group3['industry']
group3
# group2 = group0.advertiser_id.apply(list).reset_index()
# adv_ind2 = group2.advertiser_id.values
# count = 0
# for i in tqdm(range(ind.shape[0])):
#     for j in range(len(ind[i])):
#         adv_ind2[i][j] += ind[i][j]
# count  
# group2['adv_ind2'] =adv_ind2
# group2['adv_ind2'] =group2['adv_ind2'].apply(lambda x:' '.join(x))
# group2['advertiser_id'] = group2['advertiser_id'].apply(lambda x:' '.join(x))
# group2


100%|██████████| 900000/900000 [00:09<00:00, 97109.69it/s]


,user_id,industry,industry2
0,1,6 106 326 326 326 326 73 217 64 245 238 245 6,6 106 326 326 326 326 73 217 64 245 238 245 6
1,2,318 238 6 6 318 47 317 47 6 47 242 47 6 6 47 4...,318 238 6 6 318 47 317 47 6 47 242 47 6 6 47 4...
2,3,319 319 322 322 322 6 54 6 322 322 205 302 322...,319 319 322 322 322 6 54 6 322 322 205 302 322...
3,4,238 25 248 248 248 248 248 248 88 319 6 238 31...,238 25 248 248 248 248 248 248 88 319 6 238 31...
4,5,297 297 318 6 322 6 288 322 319 238 322 322 20...,297 297 318 6 322 6 288 322 319 238 322 322 20...
...,...,...,...
899995,899996,54 40 54 54 40 54 40 6 202 6 248 248 248 54,54 40 54 54 40 54 40 6 202 6 248 248 248 54
899996,899997,60 5 317 60 6 289 318 297 321 161 319 318 6 32...,60 5 317 60 6 289 318 297 321 161 319 318 6 32...
899997,899998,54 54 54 54 6 6 333 6 293 6 6 6 6 6,54 54 54 54 6 6 333 6 293 6 6 6 6 6
899998,899999,217 217 333 317 252 6 232 317 108 6 54 289 6 1...,217 217 333 317 252 6 232 317 108 6 54 289 6 1...


In [10]:
group10 = group3
group10

,user_id,industry2
0,1,6 106 326 326 326 326 73 217 64 245 238 245 6
1,2,318 238 6 6 318 47 317 47 6 47 242 47 6 6 47 4...
2,3,319 319 322 322 322 6 54 6 322 322 205 302 322...
3,4,238 25 248 248 248 248 248 248 88 319 6 238 31...
4,5,297 297 318 6 322 6 288 322 319 238 322 322 20...
...,...,...
899995,899996,54 40 54 54 40 54 40 6 202 6 248 248 248 54
899996,899997,60 5 317 60 6 289 318 297 321 161 319 318 6 32...
899997,899998,54 54 54 54 6 6 333 6 293 6 6 6 6 6
899998,899999,217 217 333 317 252 6 232 317 108 6 54 289 6 1...


In [5]:
###train set
click_ad.sort_values(by='time',inplace=True)
click_ad['creative_id'] = click_ad['creative_id'].astype(str)
click_ad['advertiser_id'] = click_ad['advertiser_id'].astype(str)
click_ad['ad_id'] = click_ad['ad_id'].astype(str)
click_ad['product_category'] = click_ad['product_category'].astype(str)
click_ad['click_times'] = click_ad['click_times'].astype(str)
click_ad['time'] = click_ad['time'].astype(str)
click_ad['uni_col'] = click_ad['advertiser_id']+'_'+ click_ad['product_id']+'_'+click_ad['industry']+'_'+click_ad['product_category']
click_ad


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,uni_col
11038879,1,43718,18612,1,19795,29,2,817,317,817_29_317_2
27435892,1,438200,1381,1,1987,26858,3,9717,60,9717_26858_60_3
8170398,1,74183,59144,1,54883,\N,18,17627,54,17627_\N_54_18
8170409,1,778236,59144,2,54883,\N,18,17627,54,17627_\N_54_18
21227515,1,24882,35447,1,34332,\N,18,17969,27,17969_\N_27_18
...,...,...,...,...,...,...,...,...,...,...
27628648,91,686189,4374309,1,3750661,3687,2,15863,247,15863_3687_247_2
22613566,91,385266,1508104,1,1311397,30888,3,16713,81,16713_30888_81_3
1297579,91,15032,2716698,1,2336903,1261,2,7115,6,7115_1261_6_2
25951973,91,857796,4026938,1,3457224,\N,5,53531,73,53531_\N_73_5


In [6]:
# click_ad.sort_values(by='time',inplace=True)

group0 = click_ad.groupby('user_id')

group1 = group0.creative_id.apply(list).reset_index()
group1['creative_id'] = group1['creative_id'].apply(lambda x:' '.join(x))


group2 = group0.uni_col.apply(list).reset_index()
group2['uni_col'] = group2['uni_col'].apply(lambda x:' '.join(x))

# group3= group0.industry.apply(list).reset_index()
# group3['industry'] = group3['industry'].apply(lambda x:' '.join(x))

# group4= group0.ad_id.apply(list).reset_index()
# group4['ad_id'] = group4['ad_id'].apply(lambda x:' '.join(x))

# group5= group0.product_id.apply(list).reset_index()
# group5['product_id'] = group5['product_id'].apply(lambda x:' '.join(x))

# group6= group0.product_category.apply(list).reset_index()
# group6['product_category'] = group6['product_category'].apply(lambda x:' '.join(x))

group3= group0.click_times.apply(list).reset_index()
group3['click_times'] = group3['click_times'].apply(lambda x:' '.join(x))

group4= group0.time.apply(list).reset_index()
group4['time'] = group4['time'].apply(lambda x:' '.join(x))
# group8= group0.adv_pro.apply(list).reset_index()
# group8['adv_pro'] = group8['adv_pro'].apply(lambda x:' '.join(x))

# group9= group0.adv_ind.apply(list).reset_index()
# group9['adv_ind'] = group9['adv_ind'].apply(lambda x:' '.join(x))

# group11= group0.uni_col.apply(list).reset_index()
# group11['uni_col'] = group11['uni_col'].apply(lambda x:' '.join(x))


group = pd.merge(group1,group2,on ='user_id' )
group = pd.merge(group,group3,on ='user_id' )
group = pd.merge(group,group4,on ='user_id' )
# group = pd.merge(group,group5,on ='user_id' )
# group = pd.merge(group,group6,on ='user_id' )
# group = pd.merge(group,group7,on ='user_id' )
# group = pd.merge(group,group8,on ='user_id' )
# group = pd.merge(group,group9,on ='user_id' )
# group = pd.merge(group,group10,on ='user_id' )
# group = pd.merge(group,group11,on ='user_id' )
group

,user_id,creative_id,uni_col,click_times,time
0,1,209778 877468 821396 1683713 122032 71691 1940...,9702_136_6_2 29455_\N_106_5 7293_\N_326_5 1466...,1 1 1 1 1 1 1 1 1 1 1 1 2,20 20 20 39 40 43 46 52 60 64 64 73 76
1,2,63441 155822 39714 609050 441462 1266180 13069...,22885_87_318_2 10686_80_238_2 18562_129_6_2 25...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...
2,3,661347 808612 593522 710859 825434 726940 3920...,32974_36256_\N_17 9877_40905_\N_17 17018_1674_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...
3,4,39588 589886 574787 1892854 1962706 2264105 12...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...
4,5,296145 350759 24333 43235 852327 1054434 12964...,11882_\N_297_5 992_\N_\N_8 22885_87_318_2 9706...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...
...,...,...,...,...,...
899995,899996,92193 654526 939409 1477926 1458800 931537 181...,18103_\N_54_18 13019_\N_40_18 9934_\N_54_18 99...,1 1 1 1 1 1 1 1 1 1 1 1 1 1,8 11 21 30 31 31 40 53 63 64 69 70 70 75
899996,899997,69204 30210 7400 60330 2804147 3011222 24333 1...,24952_26858_60_3 24975_1377_5_2 768_1400_317_2...,1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1,2 4 16 22 65 66 67 67 68 69 75 77 77 81 84 87 ...
899997,899998,648402 1251649 1370659 2302153 2915599 1779124...,18103_\N_54_12 43805_35420_\N_17 32974_39519_\...,2 1 1 1 1 1 1 1 1 1 1 1 1 1,25 40 40 50 62 63 72 73 73 75 78 78 78 91
899998,899999,307407 122834 646020 12838 774777 599699 12879...,19524_\N_217_5 8126_35593_\N_17 14930_\N_333_5...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,3 9 11 13 14 21 28 29 29 31 31 32 32 53 54 58 ...


In [7]:
group = pd.merge(group,user,on ='user_id')
# del group['gender_y']
group

,user_id,creative_id,uni_col,click_times,time,age,gender
0,1,209778 877468 821396 1683713 122032 71691 1940...,9702_136_6_2 29455_\N_106_5 7293_\N_326_5 1466...,1 1 1 1 1 1 1 1 1 1 1 1 2,20 20 20 39 40 43 46 52 60 64 64 73 76,4,1
1,2,63441 155822 39714 609050 441462 1266180 13069...,22885_87_318_2 10686_80_238_2 18562_129_6_2 25...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,10,1
2,3,661347 808612 593522 710859 825434 726940 3920...,32974_36256_\N_17 9877_40905_\N_17 17018_1674_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,7,2
3,4,39588 589886 574787 1892854 1962706 2264105 12...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,5,1
4,5,296145 350759 24333 43235 852327 1054434 12964...,11882_\N_297_5 992_\N_\N_8 22885_87_318_2 9706...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,4,1
...,...,...,...,...,...,...,...
899995,899996,92193 654526 939409 1477926 1458800 931537 181...,18103_\N_54_18 13019_\N_40_18 9934_\N_54_18 99...,1 1 1 1 1 1 1 1 1 1 1 1 1 1,8 11 21 30 31 31 40 53 63 64 69 70 70 75,5,1
899996,899997,69204 30210 7400 60330 2804147 3011222 24333 1...,24952_26858_60_3 24975_1377_5_2 768_1400_317_2...,1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1,2 4 16 22 65 66 67 67 68 69 75 77 77 81 84 87 ...,3,2
899997,899998,648402 1251649 1370659 2302153 2915599 1779124...,18103_\N_54_12 43805_35420_\N_17 32974_39519_\...,2 1 1 1 1 1 1 1 1 1 1 1 1 1,25 40 40 50 62 63 72 73 73 75 78 78 78 91,4,2
899998,899999,307407 122834 646020 12838 774777 599699 12879...,19524_\N_217_5 8126_35593_\N_17 14930_\N_333_5...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,3 9 11 13 14 21 28 29 29 31 31 32 32 53 54 58 ...,3,1


In [8]:
# group.to_csv(train_root+'train_uni.csv')
# train_ori = pd.read_csv(train_root+'train_all.csv', index_col = 0)
# group =group[['user_id','uni_col2']]
# group = pd.merge(train_ori,group,on ='user_id' )
# group
group.to_csv(train_root+'train_trf.csv')

In [12]:
click_ad['fusion'] = click_ad['product_id'] + " " + click_ad['industry'] + " " + click_ad['advertiser_id'] + " " + click_ad['product_category']
group = click_ad.groupby('user_id').fusion.apply(list).reset_index()
group['fusion'] = group['fusion'].apply(lambda x:' '.join(x))
group = pd.merge(group,user,on ='user_id')
group

,user_id,fusion,age,gender
0,1,136 120026 59722 50002 50000 120126 79475 5000...,4,1
1,2,87 120338 72905 50002 80 120258 60706 50002 12...,10,1
2,3,36256 120420 82994 50017 40905 120420 59897 50...,7,2
3,4,1862 120258 69471 50002 50000 120045 57996 500...,5,1
4,5,50000 120317 61902 50005 50000 120420 51012 50...,4,1
...,...,...,...,...
899995,899996,50000 120074 68123 50018 50000 120060 63039 50...,5,1
899996,899997,26858 120080 74972 50003 1377 120025 74995 500...,3,2
899997,899998,50000 120074 68123 50012 35420 120420 93825 50...,4,2
899998,899999,50000 120237 69544 50005 35593 120420 58146 50...,3,1


In [13]:
group.to_csv(train_root+'train_fusion.csv')

In [4]:
####test set 
# Tclick_ad.sort_values(by='time',inplace=True)
# Tclick_ad = Tclick_ad.reset_index()
# # Tclick_ad.product_id = pd.to_numeric(Tclick_ad.product_id,errors = 'coerce').fillna(50_000).map(int).map(str)
# # Tclick_ad.product_category = (Tclick_ad.product_category+50_000).map(str)
# # Tclick_ad.advertiser_id = (Tclick_ad.advertiser_id +50_020).map(str)
# # Tclick_ad.industry = (pd.to_numeric(Tclick_ad.industry ,errors = 'coerce').fillna(400).map(int)+120_020).map(str)
# Tclick_ad
# Tclick_ad['fusion'] = Tclick_ad['product_id'] + " " + Tclick_ad['industry'] + " " + Tclick_ad['advertiser_id'] + " " + Tclick_ad['product_category']
# group = Tclick_ad.groupby('user_id').fusion.apply(list).reset_index()
# group['fusion'] = group['fusion'].apply(lambda x:' '.join(x))
# group

# Tclick_ad.sort_values(by='time',inplace=True)
# Tclick_ad['product_category'] = Tclick_ad['product_category'].astype(str)
# Tclick_ad['advertiser_id'] = Tclick_ad['advertiser_id'].astype(str)
# Tclick_ad['uni_col'] = Tclick_ad['advertiser_id']+'_'+ Tclick_ad['product_id']+'_'+Tclick_ad['industry']+'_'+Tclick_ad['product_category']
# Tclick_ad

Tclick_ad.sort_values(by='time',inplace=True)
Tclick_ad['creative_id'] = Tclick_ad['creative_id'].astype(str)
Tclick_ad['advertiser_id'] = Tclick_ad['advertiser_id'].astype(str)
Tclick_ad['ad_id'] = Tclick_ad['ad_id'].astype(str)
Tclick_ad['product_category'] = Tclick_ad['product_category'].astype(str)
Tclick_ad['click_times'] = Tclick_ad['click_times'].astype(str)
Tclick_ad['time'] = Tclick_ad['time'].astype(str)
Tclick_ad['uni_col'] = Tclick_ad['advertiser_id']+'_'+ Tclick_ad['product_id']+'_'+Tclick_ad['industry']+'_'+Tclick_ad['product_category']
Tclick_ad


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,uni_col
5115271,1,3134048,171985,1,154333,1234,2,11990,321,11990_1234_321_2
7160664,1,3048376,44862,1,42085,26858,3,6776,60,6776_26858_60_3
4777789,1,3147347,27216,1,27539,\N,12,21199,54,21199_\N_54_12
7160651,1,3037569,44862,1,42085,26858,3,6776,60,6776_26858_60_3
12373892,1,3058851,111114,1,100587,42501,17,10017,\N,10017_42501_\N_17
...,...,...,...,...,...,...,...,...,...,...
26402241,91,3732006,4379687,1,3755271,643,3,10987,6,10987_643_6_3
32755821,91,3355522,4327215,1,3710832,9283,3,24987,247,24987_9283_247_3
29622320,91,3836892,4367150,1,3744662,\N,5,7133,109,7133_\N_109_5
29622315,91,3559996,4367150,1,3744662,\N,5,7133,109,7133_\N_109_5


In [5]:
# Tclick_ad.sort_values(by='time',inplace=True)
# Tclick_ad['creative_id'] = Tclick_ad['creative_id'].astype(str)
# Tclick_ad['advertiser_id'] = Tclick_ad['advertiser_id'].astype(str)
# Tclick_ad['ad_id'] = Tclick_ad['ad_id'].astype(str)
# Tclick_ad['product_category'] = Tclick_ad['product_category'].astype(str)
# Tclick_ad['click_times'] = Tclick_ad['click_times'].astype(str)

group0 = Tclick_ad.groupby('user_id')

group1 = group0.creative_id.apply(list).reset_index()
group1['creative_id'] = group1['creative_id'].apply(lambda x:' '.join(x))

group2 = group0.uni_col.apply(list).reset_index()
group2['uni_col'] = group2['uni_col'].apply(lambda x:' '.join(x))

group3= group0.click_times.apply(list).reset_index()
group3['click_times'] = group3['click_times'].apply(lambda x:' '.join(x))

group4= group0.time.apply(list).reset_index()
group4['time'] = group4['time'].apply(lambda x:' '.join(x))


group = pd.merge(group1,group2,on ='user_id' )
group = pd.merge(group,group3,on ='user_id' )
group = pd.merge(group,group4,on ='user_id' )

# group2 = group0.advertiser_id.apply(list).reset_index()
# group2['advertiser_id'] = group2['advertiser_id'].apply(lambda x:' '.join(x))

# group3= group0.industry.apply(list).reset_index()
# group3['industry'] = group3['industry'].apply(lambda x:' '.join(x))

# group4= group0.ad_id.apply(list).reset_index()
# group4['ad_id'] = group4['ad_id'].apply(lambda x:' '.join(x))

# group5= group0.product_id.apply(list).reset_index()
# group5['product_id'] = group5['product_id'].apply(lambda x:' '.join(x))

# group6= group0.product_category.apply(list).reset_index()
# group6['product_category'] = group6['product_category'].apply(lambda x:' '.join(x))

# group7= group0.click_times.apply(list).reset_index()
# group7['click_times'] = group7['click_times'].apply(lambda x:' '.join(x))

# group = pd.merge(group1,group2,on ='user_id' )
# group = pd.merge(group,group3,on ='user_id' )
# group = pd.merge(group,group4,on ='user_id' )
# group = pd.merge(group,group5,on ='user_id' )
# group = pd.merge(group,group6,on ='user_id' )
# group = pd.merge(group,group7,on ='user_id' )
group

,user_id,creative_id,uni_col,click_times,time
0,3000001,351878 103064 665090 593698 1508864 1797787 17...,7579_\N_322_18 13084_1794_248_2 7000_1701_247_...,1 1 1 1 1 1 1 1 1 1 1,3 11 11 23 29 49 49 49 54 54 82
1,3000002,152519 151984 176984 12838 72773 64667 81234 7...,12993_1674_322_2 27800_\N_24_18 24661_1687_5_2...,1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 ...,9 10 12 12 15 17 23 23 24 25 26 26 30 30 31 31...
2,3000003,161840 367084 36634 115761 73137 150407 41212 ...,22367_82_319_2 40491_34504_202_5 22338_2065_23...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,1 3 3 3 3 4 4 11 11 15 19 21 27 37 38 43 55 58...
3,3000004,108656 849706 678427 9870 157180 94025 907546 ...,8520_1036_319_2 28323_\N_40_18 2286_\N_54_18 8...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,13 15 16 18 18 23 27 32 35 40 40 40 40 62 63 6...
4,3000005,123860 183003 40625 26793 71219 259607 167448 ...,30710_1896_238_2 29243_1987_26_2 23746_129_6_2...,1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,2 3 6 7 7 8 9 13 26 27 28 33 35 38 51 55 62 65...
...,...,...,...,...,...
999995,3999996,172004 237425 54100 417864 282643 30856 116356...,10985_1261_6_2 7725_1404_2_2 10987_1261_6_2 35...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 2 ...,1 1 2 5 8 10 12 24 28 28 30 34 35 35 40 40 40 ...
999996,3999997,271727 390953 914417 641757 50445 14993 793940...,10986_\N_6_12 16639_1246_81_2 15102_\N_222_5 3...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,2 10 17 18 18 20 30 30 45 47 48 57 59 65 67 82...
999997,3999998,100456 464055 54150 316582 882346 882346 77124...,23365_\N_27_18 28073_2344_167_2 10989_\N_202_1...,1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1,4 9 12 15 17 22 25 32 39 46 51 61 65 68 68 75 ...
999998,3999999,273715 95215 146101 117043 840 366940 148209 7...,37660_\N_322_18 22626_1062_238_2 20528_1492_32...,1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 ...,2 2 2 3 6 7 8 12 19 20 21 26 26 26 26 29 30 31...


In [3]:
Tclick_log.sort_values(by='time',inplace=True)
Tclick_log['creative_id'] = Tclick_log['creative_id'].astype(str)
group = Tclick_log.groupby('user_id').creative_id.apply(list).reset_index()
group['creative_id'] = group['creative_id'].apply(lambda x:' '.join(x))
group

,user_id,creative_id
0,3000001,351878 665090 103064 593698 1508864 1711578 22...
1,3000002,152519 151984 12838 176984 72773 64667 81234 7...
2,3000003,161840 73137 115761 36634 367084 150407 41212 ...
3,3000004,108656 849706 678427 9870 157180 94025 907546 ...
4,3000005,123860 183003 40625 26793 71219 259607 167448 ...
...,...,...
999995,3999996,237425 172004 54100 417864 282643 30856 116356...
999996,3999997,271727 390953 914417 50445 641757 14993 793940...
999997,3999998,100456 464055 54150 316582 882346 882346 77124...
999998,3999999,95215 273715 146101 117043 840 366940 148209 7...


In [6]:
group.to_csv(test_root+'test_trf.csv')
# test_group = group
# # test_group = pd.read_csv(test_root+'test_creative.csv',index_col=0)
# test_group

In [2]:
train_root1 = './data/train_preliminary/'
train_root2 = './data/train_semi_final/'
test_root = './data/test/'
ad1 = pd.read_csv(train_root1+'ad.csv')
click_log1 = pd.read_csv(train_root1 +'click_log.csv')
user1 = pd.read_csv(train_root1+'user.csv')

ad2 = pd.read_csv(train_root2+'ad.csv')
click_log2 = pd.read_csv(train_root2 +'click_log.csv')
user2 = pd.read_csv(train_root2+'user.csv')

Tclick_log = pd.read_csv(test_root +'click_log.csv')
Tad = pd.read_csv(test_root +'ad.csv')

In [3]:
user = user1.append(user2)
click_ad1 = pd.merge(click_log1, ad1, on='creative_id')
click_ad2 = pd.merge(click_log2, ad2, on='creative_id')
click_ad = click_ad1.append(click_ad2)
# click_log.creative_id = click_log.creative_id.astype(str)
click_ad

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319
...,...,...,...,...,...,...,...,...,...
70210157,48,2137713,2347217,1,2023744,1062,2,36090,238
70210158,70,2137719,3359734,1,2890881,21631,8,58583,329
70210159,52,2137721,2522001,1,2171997,2701,2,6759,248
70210160,69,2137721,3394457,1,2921168,2819,2,2466,253


In [8]:
Tclick_ad = pd.merge(Tclick_log, Tad, on='creative_id')
Tclick_ad

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,10,3142948,645764,1,573314,58,2,14689,6
2,14,3170643,645764,1,573314,58,2,14689,6
3,10,3194257,645764,1,573314,58,2,14689,6
4,21,3222925,645764,1,573314,58,2,14689,6
...,...,...,...,...,...,...,...,...,...
33585507,49,3131983,2033604,1,1757820,\N,5,29292,109
33585508,4,3131984,187695,1,168395,1634,2,14494,322
33585509,5,3131984,322785,1,290114,2031,2,19976,238
33585510,16,3131984,900585,1,793805,1766,2,33097,319


In [7]:
###train set
click_ad.sort_values(by='time',inplace=True)
click_ad['creative_id'] = click_ad['creative_id'].astype(str)
click_ad['advertiser_id'] = click_ad['advertiser_id'].astype(str)
click_ad['ad_id'] = click_ad['ad_id'].astype(str)
click_ad['product_category'] = click_ad['product_category'].astype(str)
click_ad['click_times'] = click_ad['click_times'].astype(str)
click_ad['time'] = click_ad['time'].astype(str)
click_ad['uni_col'] = click_ad['advertiser_id']+'_'+ click_ad['product_id']+'_'+click_ad['industry']+'_'+click_ad['product_category']
click_ad

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,uni_col
3636476,1,437774,62274,1,57709,101,2,885,183,885_101_183_2
25136749,1,2061,163641,1,146744,\N,18,19750,73,19750_\N_73_18
25136748,1,201700,163641,1,146744,\N,18,19750,73,19750_\N_73_18
25136747,1,125240,163641,1,146744,\N,18,19750,73,19750_\N_73_18
25136746,1,751764,163641,2,146744,\N,18,19750,73,19750_\N_73_18
...,...,...,...,...,...,...,...,...,...,...
27048667,91,56457,4444265,1,3810887,20350,8,54310,86,54310_20350_86_8
21588562,91,148044,1586236,1,1376876,30709,3,5502,231,5502_30709_231_3
27048660,91,56457,2951731,1,2538032,20484,8,55909,329,55909_20484_329_8
29698068,91,681478,4290314,1,3679791,38753,17,16145,\N,16145_38753_\N_17


In [8]:
group0 = click_ad.groupby('user_id')

group1 = group0.creative_id.apply(list).reset_index()
group1['creative_id'] = group1['creative_id'].apply(lambda x:' '.join(x))
group2 = group0.uni_col.apply(list).reset_index()
group2['uni_col'] = group2['uni_col'].apply(lambda x:' '.join(x))
group3= group0.click_times.apply(list).reset_index()
group3['click_times'] = group3['click_times'].apply(lambda x:' '.join(x))
group4= group0.time.apply(list).reset_index()
group4['time'] = group4['time'].apply(lambda x:' '.join(x))


group = pd.merge(group1,group2,on ='user_id' )
group = pd.merge(group,group3,on ='user_id' )
group = pd.merge(group,group4,on ='user_id' )

group

,user_id,creative_id,uni_col,click_times,time
0,1,877468 821396 209778 1683713 122032 1940159 90...,29455_\N_106_5 7293_\N_326_5 9702_136_6_2 1466...,1 1 1 1 1 1 1 1 1 1 1 2,20 20 20 39 40 46 52 60 64 64 73 76
1,2,155822 1266180 1657530 1074235 1252062 1662244...,10686_80_238_2 34505_\N_47_18 918_1261_6_2 172...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,11 28 38 41 42 42 49 49 51 53 53 53 55 59 60 6...
2,3,661347 808612 825434 710859 593522 726940 3920...,32974_36256_\N_17 9877_40905_\N_17 14186_35985...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...
3,4,39588 589886 574787 1892854 1962706 2264105 31...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...
4,5,296145 350759 43235 852327 1054434 1296456 124...,11882_\N_297_5 992_\N_\N_8 9706_136_6_2 38760_...,1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,3 13 15 20 21 24 25 27 28 29 30 32 32 35 35 48...
...,...,...,...,...,...
899910,899996,654526 939409 1477926 931537 1458800 1814717 3...,13019_\N_40_18 9934_\N_54_18 9934_\N_54_18 228...,1 1 1 1 1 1 1 1 1 1 1,11 21 30 31 31 40 63 69 70 70 75
899911,899997,60330 2804147 3011222 1879314 177415 127573 36...,28874_26858_60_3 14681_\N_6_18 21185_\N_289_5 ...,1 1 2 1 1 1 1 1 1 1 1 1 1,22 65 66 67 68 69 75 77 81 84 87 87 87
899912,899998,1251649 2302153 2915599 1779124 71752 3534396 ...,43805_35420_\N_17 51814_43559_\N_17 14681_\N_6...,1 1 1 1 1 1 1 1 1 1 1 1,40 50 62 63 72 73 73 75 78 78 78 91
899913,899999,307407 122834 646020 774777 1287932 1513243 45...,19524_\N_217_5 8126_35593_\N_17 14930_\N_333_5...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,3 9 11 14 28 29 31 32 32 53 54 72 75 83 83


In [9]:
group = pd.merge(group,user,on ='user_id')
group

,user_id,creative_id,uni_col,click_times,time,age,gender
0,1,877468 821396 209778 1683713 122032 1940159 90...,29455_\N_106_5 7293_\N_326_5 9702_136_6_2 1466...,1 1 1 1 1 1 1 1 1 1 1 2,20 20 20 39 40 46 52 60 64 64 73 76,4,1
1,2,155822 1266180 1657530 1074235 1252062 1662244...,10686_80_238_2 34505_\N_47_18 918_1261_6_2 172...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,11 28 38 41 42 42 49 49 51 53 53 53 55 59 60 6...,10,1
2,3,661347 808612 825434 710859 593522 726940 3920...,32974_36256_\N_17 9877_40905_\N_17 14186_35985...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,7,2
3,4,39588 589886 574787 1892854 1962706 2264105 31...,19451_1862_238_2 7976_\N_25_18 13084_2625_248_...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,5,1
4,5,296145 350759 43235 852327 1054434 1296456 124...,11882_\N_297_5 992_\N_\N_8 9706_136_6_2 38760_...,1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,3 13 15 20 21 24 25 27 28 29 30 32 32 35 35 48...,4,1
...,...,...,...,...,...,...,...
899910,899996,654526 939409 1477926 931537 1458800 1814717 3...,13019_\N_40_18 9934_\N_54_18 9934_\N_54_18 228...,1 1 1 1 1 1 1 1 1 1 1,11 21 30 31 31 40 63 69 70 70 75,5,1
899911,899997,60330 2804147 3011222 1879314 177415 127573 36...,28874_26858_60_3 14681_\N_6_18 21185_\N_289_5 ...,1 1 2 1 1 1 1 1 1 1 1 1 1,22 65 66 67 68 69 75 77 81 84 87 87 87,3,2
899912,899998,1251649 2302153 2915599 1779124 71752 3534396 ...,43805_35420_\N_17 51814_43559_\N_17 14681_\N_6...,1 1 1 1 1 1 1 1 1 1 1 1,40 50 62 63 72 73 73 75 78 78 78 91,4,2
899913,899999,307407 122834 646020 774777 1287932 1513243 45...,19524_\N_217_5 8126_35593_\N_17 14930_\N_333_5...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,3 9 11 14 28 29 31 32 32 53 54 72 75 83 83,3,1


In [10]:
group.to_csv(train_root2+'train_trf_drop.csv')

In [9]:
####test set 
Tclick_ad.sort_values(by='time',inplace=True)
Tclick_ad['creative_id'] = Tclick_ad['creative_id'].astype(str)
Tclick_ad['advertiser_id'] = Tclick_ad['advertiser_id'].astype(str)
Tclick_ad['ad_id'] = Tclick_ad['ad_id'].astype(str)
Tclick_ad['product_category'] = Tclick_ad['product_category'].astype(str)
Tclick_ad['click_times'] = Tclick_ad['click_times'].astype(str)
Tclick_ad['time'] = Tclick_ad['time'].astype(str)
Tclick_ad['uni_col'] = Tclick_ad['advertiser_id']+'_'+ Tclick_ad['product_id']+'_'+Tclick_ad['industry']+'_'+Tclick_ad['product_category']
Tclick_ad

KeyboardInterrupt: 

In [ ]:
group0 = Tclick_ad.groupby('user_id')

group1 = group0.creative_id.apply(list).reset_index()
group1['creative_id'] = group1['creative_id'].apply(lambda x:' '.join(x))

group2 = group0.uni_col.apply(list).reset_index()
group2['uni_col'] = group2['uni_col'].apply(lambda x:' '.join(x))
                                            
                                            
group3 = group0.click_times.apply(list).reset_index()
group3['click_times'] = group3['click_times'].apply(lambda x:' '.join(x))
                                            
group4 = group0.time.apply(list).reset_index()
group4['time'] = group4['time'].apply(lambda x:' '.join(x))


group = pd.merge(group1,group2,on ='user_id' )
group = pd.merge(group,group3,on ='user_id' )
group = pd.merge(group,group4,on ='user_id' )


group